In [1]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import cv2


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [3]:
mtcnn = MTCNN(keep_all=True, device=device,select_largest=False)

In [5]:
cap=cv2.VideoCapture("C:/Users/user/Desktop/New_Sample/원천데이터/lip_J_1_F_02_C032_A_010.mp4")


width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count=cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps=cap.get(cv2.CAP_PROP_FPS)

print('가로:',str(width))
print('세로:',str(height))
print('총 프레임수:',str(count))
print('FPS:',str(fps))


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('lip_facenet.mp4', fourcc, fps, (96, 96))


if not cap.isOpened():
    print("Could not open webcam")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    test_frame=frame.copy()
    test_frame=cv2.resize(test_frame,dsize=(540,960))
    boxes,prob,points=mtcnn.detect(test_frame,landmarks=True)
    
    frame_draw =test_frame.copy()
    
    if boxes is None:
        print("얼굴 인식에 실패하였습니다")
        continue
        
    boxes=boxes.astype(int)
    points=points.astype(int)

    max_box=prob.argmax()
    crop_frame=frame_draw[boxes[max_box][1]:boxes[max_box][3],boxes[max_box][0]:boxes[max_box][2]]
    
    
    ly = (points[0][3][1]+points[0][3][1])//2
    y = boxes[max_box][3]-boxes[max_box][1]
    x = boxes[max_box][2]-boxes[max_box][0]
    
    lip = frame_draw[ly-y//8:ly+y//8, points[0][3][0]-x//10:points[0][4][0]+x//10]


    lip=cv2.resize(lip,dsize=(96,96))
    
    if ret:
        
        cv2.imshow("test", frame)
        cv2.imshow('crop_face',crop_frame)
        out.write(lip)
        cv2.imshow('lip',lip)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

가로: 1920.0
세로: 1080.0
총 프레임수: 9129.0
FPS: 29.97002997002997
